In [9]:
# MODELO FINAL SEPARA EN 6 COLUMNAS (FECHA, HORA, TITULO DE NOTICIA, AUTOR, TEMA, RESUMEN)
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive

# Montamos Google Drive
drive.mount('/content/drive')

# Define la URL base y las fechas de inicio y fin
url_base = 'https://elcomercio.pe/archivo/todas/'
fecha_inicio = '2023-08-31' # '2022-09-01'
fecha_fin = '2023-09-30' # '2023-09-30'

# Lista para almacenar los DataFrames de cada fecha
dfs = []

# Recorre las fechas desde fecha_inicio hasta fecha_fin
current_fecha = fecha_inicio
while current_fecha <= fecha_fin:
    # Construye la URL con la fecha actual
    url = f'{url_base}{current_fecha}/'

    # Realizamos una solicitud GET a la página web
    response = requests.get(url)

    # Verificamos si la solicitud fue exitosa
    if response.status_code == 200:
        content = response.text

        # Parsear el contenido HTML utilizando BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')

        # Encontrar todo el texto relevante
        fecha_elements = soup.find_all('span', class_='story-item__date-time')
        hora_elements = soup.find_all('span', class_='story-item__date-time')
        titulo_elements = soup.find_all('h2', class_='story-item__content-title')
        autor_elements = soup.find_all('a', class_='story-item__author block uppercase mt-10 font-thin text-xs text-gray-200')
        tema_elements = soup.find_all('a', class_='story-item__section story-item__section--desktop text-sm text-black md:mb-15 hidden')
        resumen_elements = soup.find_all('p', class_='story-item__subtitle')

        # Extraer el texto de los elementos encontrados
        fechas = [fecha.text for fecha in fecha_elements[::2]]
        horas = [hora.text for hora in hora_elements[1::2]]
        titulos = [titulo.find('a').text for titulo in titulo_elements if titulo.find('a')]
        autores = [autor.text for autor in autor_elements]
        temas = [tema.text for tema in tema_elements]
        resumenes = [resumen.text for resumen in resumen_elements]

        # Asegurarse de que ambas listas tengan la misma longitud
        min_length = min(len(fechas), len(horas), len(titulos), len(autores))

        fechas = fechas[:min_length]
        horas = horas[:min_length]
        titulos = titulos[:min_length]
        autores = autores[:min_length]

        # Crear un DataFrame de pandas con el texto en columnas
        df = pd.DataFrame({'Fecha': fechas, 'Hora': horas, 'Titulo de noticia': titulos, 'Autor': autores, 'Tema': temas, 'Resumen': resumenes})

        # Agregar el DataFrame a la lista
        dfs.append(df)
    else:
        print(f"No se pudo acceder a la página web para la fecha {current_fecha}.")

    # Avanzar a la siguiente fecha
    current_fecha = pd.to_datetime(current_fecha) + pd.DateOffset(days=1)
    current_fecha = current_fecha.strftime('%Y-%m-%d')

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)

# Especifica la ruta completa del archivo CSV en tu Google Drive
csv_filepath = '/content/drive/My Drive/scrapy/ScrapingTexto.csv'

# Guardar el DataFrame en un archivo CSV en Google Drive
final_df.to_csv(csv_filepath, index=False, escapechar='\\')

print(f"Se ha extraído y guardado el texto en '{csv_filepath}'.")
print(len(final_df))

Mounted at /content/drive
Se ha extraído y guardado el texto en '/content/drive/My Drive/scrapy/ScrapingTexto.csv'.
3100
